In [1]:
%pip install -q openai 
%pip install python-dotenv
%pip install torch torchvision
%pip install pandas numpy matplotlib 
%pip install imageio
%pip install certifi
%pip install opencv-python moviepy pillow


[notice] A new release of pip is available: 23.1.2 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.1.1
[notice] To update, run: pip3 install 

In [61]:
import os
from openai import OpenAI

from dotenv import load_dotenv

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

import imageio

from openai import OpenAI 
import cv2
from moviepy.editor import VideoFileClip
import moviepy.editor as mp
import time
import base64
import json
from io import BytesIO
from collections import defaultdict



In [3]:
load_dotenv()
openai_api_key = os.getenv("API_KEY")

MODEL = "gpt-4o"

client = OpenAI(
    api_key=openai_api_key
)

# Preprocessing gif
converting .gif to .png by processing it into frames

In [ ]:
# def extract_frames_from_gif(gif_path, output_folder):
#     reader = imageio.get_reader(gif_path)
#     for i, frame in enumerate(reader):
#         frame_path = os.path.join(output_folder, f"frame_{i}.png")
#         imageio.imwrite(frame_path, frame)

# def preprocess_gif_dataset(dataset_path):
#     for subdir, dirs, files in os.walk(dataset_path):
#         for file in files:
#             filepath = os.path.join(subdir, file)
#             if filepath.endswith('.gif'):
#                 output_folder = os.path.join(subdir, os.path.splitext(file)[0])  # Create a folder for each GIF
#                 os.makedirs(output_folder, exist_ok=True)
#                 extract_frames_from_gif(filepath, output_folder)

# # Apply preprocessing
# preprocess_gif_dataset('mgif/moving-gif/train')
# preprocess_gif_dataset('mgif/moving-gif/test')

converting gif frams to JSON for fine-tuning

In [60]:
# Function to process a video and extract frames at regular intervals
def process_video(video_path, seconds_per_frame=2):
    base64Frames = []
    base_video_path, _ = os.path.splitext(video_path)

    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video.get(cv2.CAP_PROP_FPS)
    frames_to_skip = int(fps * seconds_per_frame)
    curr_frame = 0

    # Loop through the video and extract frames at specified sampling rate
    while curr_frame < total_frames - 1:
        video.set(cv2.CAP_PROP_POS_FRAMES, curr_frame)
        success, frame = video.read()
        if not success:
            break
        _, buffer = cv2.imencode(".jpg", frame)
        base64Frames.append(base64.b64encode(buffer).decode("utf-8"))
        curr_frame += frames_to_skip
    video.release()

    return base64Frames

# Function to process PNG frames and convert them to base64
def process_png_frames(dir_path, seconds_per_frame=2):
    base64Frames = []
    frames = sorted([f for f in os.listdir(dir_path) if f.endswith('.png')])
    frames_to_skip = int(1 / seconds_per_frame)  # Assuming 1 frame per second for simplicity

    for i, frame in enumerate(frames):
        if i % frames_to_skip == 0:
            frame_path = os.path.join(dir_path, frame)
            image = imageio.imread(frame_path)
            with BytesIO() as buffer:
                imageio.imwrite(buffer, image, format='png')
                base64Frames.append(base64.b64encode(buffer.getvalue()).decode('utf-8'))
    return base64Frames

# Function to generate fine-tuning data
def generate_finetuning_data(dataset_path, max_folders):
    examples = []

    for subdir, dirs, files in os.walk(dataset_path):
        for dir in dirs:
            dir_number = dir.lstrip("0")
            if dir_number and int(dir_number) < max_folders:
                dir_path = os.path.join(subdir, dir)
                base64Frames = process_png_frames(dir_path, seconds_per_frame=1)
                user_content = [
                    "These are the frames from the video.",
                    *map(lambda x: {"type": "image_url", "image_url": {"url": f'data:image/png;base64,{x}', "detail": "low"}}, base64Frames)
                ]
                assistant_content = "MGIF"
                example = {
                    "messages": [
                        {"role": "system", "content": """You are a classifier. Classify the provided video into one of the following categories: Vox (people talking), TEDTalk (standing talking), TaiChi, MGIF Respond with only category (one word)."""},
                        {"role": "user", "content": user_content},
                        {"role": "assistant", "content": assistant_content}
                    ]
                }
                examples.append(example)

    return examples

# Generate fine-tuning data
train_data = generate_finetuning_data('mgif/moving-gif/train', max_folders=120)
test_data = generate_finetuning_data('mgif/moving-gif/test', max_folders=30)

print(f"Generated {len(train_data)} training examples")
print(f"Generated {len(test_data)} testing examples")

# Save the generated data to JSON files
with open('mgif/moving-gif/train_finetuning_data.json', 'w') as train_file:
    json.dump(train_data, train_file, indent=4)

with open('mgif/moving-gif/test_finetuning_data.json', 'w') as test_file:
    json.dump(test_data, test_file, indent=4)


Generated 20 training examples
Generated 29 testing examples


# JSON Format validation
Checking JSON format before fine-tunging

In [66]:
import json
from collections import defaultdict
import base64

# Function to check if a string is a valid base64 encoded string
def is_base64(sb):
    try:
        if isinstance(sb, str):
            sb_bytes = bytes(sb, 'utf-8')
        elif isinstance(sb, bytes):
            sb_bytes = sb
        else:
            raise ValueError("Input should be a string or bytes")
        return base64.b64encode(base64.b64decode(sb_bytes)) == sb_bytes
    except Exception:
        return False

# Function to validate JSON data format
def validate_json_data(json_data):
    format_errors = defaultdict(int)

    for ex in json_data:
        if not isinstance(ex, dict):
            format_errors["data_type"] += 1
            continue

        messages = ex.get("messages", None)
        if not messages:
            format_errors["missing_messages_list"] += 1
            continue

        for message in messages:
            if "role" not in message or "content" not in message:
                format_errors["message_missing_key"] += 1

            if any(k not in ("role", "content", "name", "function_call", "weight", "type", "image_url") for k in message):
                format_errors["message_unrecognized_key"] += 1

            if message.get("role", None) not in ("system", "user", "assistant", "function"):
                format_errors["unrecognized_role"] += 1

            content = message.get("content", None)
            function_call = message.get("function_call", None)

            if message.get("role") == "user" and isinstance(content, list):
                for item in content:
                    if isinstance(item, dict) and "image_url" in item:
                        image_url = item["image_url"].get("url", "")
                        if not is_base64(image_url.split(",")[1]):
                            format_errors["invalid_base64_image"] += 1
            elif not content and not function_call:
                format_errors["missing_content"] += 1

        if not any(message.get("role", None) == "assistant" for message in messages):
            format_errors["example_missing_assistant_message"] += 1

    return format_errors


# Data Warnings and Token Counts

In [67]:
# Function to count tokens (example placeholder functions, replace with actual implementation)
def num_tokens_from_messages(messages):
    return sum(len(message['content']) for message in messages if 'content' in message)

def num_assistant_tokens_from_messages(messages):
    return sum(len(message['content']) for message in messages if 'role' in message and message['role'] == 'assistant')

# Function to print distribution
def print_distribution(data, description):
    from collections import Counter
    counter = Counter(data)
    print(f"\nDistribution of {description}:")
    for k, v in sorted(counter.items()):
        print(f"{k}: {v}")

# Load and validate JSON data
def process_json_data(file_path):
    with open(file_path) as f:
        data = json.load(f)

    format_errors = validate_json_data(data)
    
    n_missing_system = 0
    n_missing_user = 0
    n_messages = []
    convo_lens = []
    assistant_message_lens = []

    for ex in data:
        messages = ex["messages"]
        if not any(message["role"] == "system" for message in messages):
            n_missing_system += 1
        if not any(message["role"] == "user" for message in messages):
            n_missing_user += 1
        n_messages.append(len(messages))
        convo_lens.append(num_tokens_from_messages(messages))
        assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

    print(f"\nProcessing {file_path}")
    if format_errors:
        print("Found format errors:")
        for k, v in format_errors.items():
            print(f"  {k}: {v}")
    else:
        print("No format errors found")

    print("Num examples missing system message:", n_missing_system)
    print("Num examples missing user message:", n_missing_user)
    print_distribution(n_messages, "num_messages_per_example")
    print_distribution(convo_lens, "num_total_tokens_per_example")
    print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
    n_too_long = sum(l > 16385 for l in convo_lens)
    print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

# Process training data
process_json_data('mgif/moving-gif/train_finetuning_data.json')

# Process testing data
process_json_data('mgif/moving-gif/test_finetuning_data.json')



Processing mgif/moving-gif/train_finetuning_data.json
No format errors found
Num examples missing system message: 0
Num examples missing user message: 0

Distribution of num_messages_per_example:
3: 20

Distribution of num_total_tokens_per_example:
198: 1
199: 1
200: 1
202: 3
203: 2
204: 1
205: 1
206: 2
207: 1
208: 1
209: 1
210: 1
211: 1
212: 1
220: 1
228: 1

Distribution of num_assistant_tokens_per_example:
4: 20

0 examples may be over the 16,385 token limit, they will be truncated during fine-tuning

Processing mgif/moving-gif/test_finetuning_data.json
No format errors found
Num examples missing system message: 0
Num examples missing user message: 0

Distribution of num_messages_per_example:
3: 29

Distribution of num_total_tokens_per_example:
196: 1
200: 1
201: 2
202: 7
203: 2
205: 2
206: 4
208: 1
209: 1
211: 1
218: 1
219: 2
244: 1
264: 1
314: 1
374: 1

Distribution of num_assistant_tokens_per_example:
4: 29

0 examples may be over the 16,385 token limit, they will be truncated du

# Process input video

In [17]:

VIDEO_PATH = "tedtalk/2024-catie-cuan-001-2590b41f-d2e2-4469-b090-5000k.mp4"

def process_video(video_path, seconds_per_frame=2):
    base64Frames = []
    base_video_path, _ = os.path.splitext(video_path)

    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video.get(cv2.CAP_PROP_FPS)
    frames_to_skip = int(fps * seconds_per_frame)
    curr_frame = 0

    # Loop through the video and extract frames at specified sampling rate
    while curr_frame < total_frames - 1:
        video.set(cv2.CAP_PROP_POS_FRAMES, curr_frame)
        success, frame = video.read()
        if not success:
            break
        _, buffer = cv2.imencode(".jpg", frame)
        base64Frames.append(base64.b64encode(buffer).decode("utf-8"))
        curr_frame += frames_to_skip
    video.release()

    # Extract audio from video if audio track is present
    audio_path = None
    clip = VideoFileClip(video_path)
    if clip.audio is not None:
        audio_path = f"{base_video_path}.mp3"
        clip.audio.write_audiofile(audio_path, bitrate="32k")
        clip.audio.close()

    clip.close()

    print(f"Extracted {len(base64Frames)} frames")
    if audio_path:
        print(f"Extracted audio to {audio_path}")
    else:
        print("No audio track found in the video")

    return base64Frames, audio_path

# Extract 1 frame per second. You can adjust the `seconds_per_frame` parameter to change the sampling rate
base64Frames, audio_path = process_video(VIDEO_PATH, seconds_per_frame=1)

transcription_text = ""
if audio_path:
    transcription = client.audio.transcriptions.create(
        model="whisper-1",
        file=open(audio_path, "rb"),
    )
    transcription_text = transcription.text
else:
    transcription_text = "No audio track available to transcribe."

## Generate a summary with visual and audio and classify the video
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": """You are generating a video summary. Create a summary of the provided video and its transcript. Respond in Markdown."""},
        {"role": "user", "content": [
            "These are the frames from the video.",
            *map(lambda x: {"type": "image_url", 
                            "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}}, base64Frames),
            {"type": "text", "text": f"The audio transcription is: {transcription_text}"}
            ],
        }
    ],
    temperature=0,
)

# Print summary response
print(response.choices[0].message.content)

## Categorize the video
categorization_response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": """You are a classifier. Classify the provided video into one of the following categories: Vox (people talking), TEDTalk (standing talking), TaiChi, MGIF."""},
        {"role": "user", "content": [
            "These are the frames from the video.",
            *map(lambda x: {"type": "image_url", 
                            "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}}, base64Frames),
            {"type": "text", "text": f"The audio transcription is: {transcription_text}"}
            ],
        }
    ],
    temperature=0,
)

# Print categorization response
print(categorization_response.choices[0].message.content)


MoviePy - Writing audio in tedtalk/2024-catie-cuan-001-2590b41f-d2e2-4469-b090-5000k.mp3


MoviePy - Done.
Extracted 10 frames
Extracted audio to tedtalk/2024-catie-cuan-001-2590b41f-d2e2-4469-b090-5000k.mp3


### Video Summary

The video features a speaker delivering a talk on a stage with a red curtain backdrop, characteristic of a TED Talk setting. The speaker is dressed in a blue suit and is holding a clicker, indicating the use of visual aids or slides during the presentation.

#### Key Points from the Transcript:
- The speaker discusses the application of human interaction and expression to robots.
- A future scenario involving a robot is introduced, suggesting a focus on the integration of human-like qualities in robotic technology.

#### Visual Elements:
- The speaker is seen in various frames, gesturing and engaging with the audience.
- The TED logo is prominently displayed on the stage, reinforcing the formal and educational context of the talk.
- The audience is visible in some frames, indicating a live presentation.

#### Overall Theme:
The talk appears to explore the intersection of human behavior and robotics, emphasizing how human-like interactions can enhance the functionalit

In [20]:
## Categorize the video
categorization_response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": """You are a classifier. Classify the provided video into one of the following categories: Vox (people talking), TEDTalk (standing talking), TaiChi, MGIF Respond with only category (one word)."""},
        {"role": "user", "content": [
            "These are the frames from the video.",
            *map(lambda x: {"type": "image_url", 
                            "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}}, base64Frames),
            {"type": "text", "text": f"The audio transcription is: {transcription_text}"}
            ],
        }
    ],
    temperature=0,
)

# Extract the classification result and store it in a variable
category = categorization_response.choices[0].message.content.strip()

# Print categorization response
print(category)

TEDTalk
